## Overview

[Palmyra Creative](https://writer.com/llms/palmyra-creative/) is Writer’s LLM engineered to elevate creative thinking and writing across diverse professional contexts. You can use it for a wide range of creative tasks, from brainstorming to crafting compelling narratives.

This cookbook demonstrates how to use Palmyra Creative with the [text generation](https://dev.writer.com/api-guides/api-reference/completion-api/text-generation) and [chat completions](https://dev.writer.com/api-guides/api-reference/completion-api/chat-completion) APIs.

## Prerequisites

- An [AI Studio](https://aap.writer.com) account. Sign up for a [free account here](https://aap.writer.com/register).
- A Writer API key. See how to [generate an API key here](https://dev.writer.com/api-guides/quickstart).

## Setup

First, install the Writer SDK:

In [ ]:
pip install -qU writer-sdk


Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()

## Usage

### Imaginative brainstorming

The following example demonstrates how to use Palmyra Creative for brainstorming ideas within specific constraints. It uses the [text generation](https://dev.writer.com/api-guides/api-reference/completion-api/text-generation) API and streams the response.

In [ ]:
prompt = """
Design a sustainable and eco-friendly line of outdoor gear for the next generation of environmentally conscious hikers and campers.
Products must be made from at least 80% recycled or biodegradable materials. Products must be priced competitively with existing products in the market ($50-$200).
Generate 5-7 unique product ideas.
Consider incorporating technology, such as integrated solar panels, GPS tracking, or other features that enhance the outdoor experience.
"""

response = client.completions.create(model="palmyra-creative", prompt=prompt, stream=True)

for chunk in response:
    print(chunk.value, end="", flush=True)

### Enhanced storytelling

The next example demonstrates how to use Palmyra Creative for storytelling using the text generation API.

In [ ]:
prompt = """
Introduce the latest innovation in road cycling. Here is a beginning of an abstract continue it:
The "AeroCore" wheelset, revolutionizing the way you cut through the air with cutting-edge carbon technology.
Featuring an advanced design that combines aero-optimized rim profiles with a unique carbon fiber layup,
the AeroCore promises to shave off precious seconds from your ride times.
"""

response = client.completions.create(model="palmyra-creative", prompt=prompt, stream=True)

for chunk in response:
    print(chunk.value, end="", flush=True)

### Use Palmyra Creative in a chat

The following example demonstrates how to use Palmyra Creative in a chat with a general Palmyra LLM. It uses the [chat completions](https://dev.writer.com/api-guides/model-delegation) API and passes the `palmyra-creative` model as a tool that the LLM can use when a user asks for creative ideas.

First, define the tool that will use Palmyra Creative:


In [ ]:
tools = [{
    "type": "llm",
    "function": {
        "description": "A function that will invoke the llm identified by the given model, specialized in creative thinking and writing. Any user request asking for creative ideas or brainstorming should use this tool.",
        "model": "palmyra-creative"
    }
}]

Then, pass the `tools` array when creating a chat completion. The example below creates a hypothetical chat with a user who asks for creative ideas for a new product. It streams the response and also displays the value from the `llm_data` field of the response, which shows the tool call used to generate the response.

In [ ]:
messages = [{"role": "user", "content": "Help generate five ideas for the name of a new SMS-based product that helps people learn to code via text messages."}]

response = client.chat.chat(
    model="palmyra-x-004", 
    messages=messages, 
    tools=tools,  # The tools array defined earlier.
    tool_choice="auto",
    stream=True,
    max_tokens=1064
)

llm_data = None
for chunk in response:
    if chunk.choices[0].delta.llm_data is not None:
        llm_data = chunk.choices[0].delta.llm_data
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="", flush=True)

print("\nLLM data:", llm_data)
